In [11]:
from google.cloud import storage

import vertexai
from vertexai.generative_models import GenerativeModel, Part
from google.adk.tools import ToolContext

# Initialize Vertex AI once
vertexai.init()

In [ ]:


def get_spec(spec_name):
    gcs_uri = f'gs://cob-agent-sandbox/schreiber-specs/{spec_name} Spec Sheet.pdf'
    print(gcs_uri)
    model = GenerativeModel("gemini-2.5-flash")

    # Create a Part object directly from the GCS URI
    pdf_file = Part.from_uri(
        mime_type="application/pdf",
        uri=gcs_uri
    )

    # The prompt for the model
    prompt = "Please provide a concise summary of this document."

    # Construct the full request and generate the content
    response = model.generate_content([pdf_file, prompt])

    return  {"status": "success", "summary": response.text}


In [23]:
get_spec('ES788')

gs://cob-agent-sandbox/schreiber-specs/ES788 Spec Sheet.pdf


{'status': 'success',
 'summary': 'This document is a spec sheet for Product No. ES788. It outlines the quality requirements for the product, including:\n*   **Appearance:** Clear, colorless, with no observable particulate.\n*   **Color (APHA):** Maximum of 10.\n*   **Specific Gravity @20°C:** Within the range of 0.9114-0.9246.'}

In [ ]:
def extract_product_no(gcs_uri: str) -> str:
    """
    Extracts the product number from a PDF document stored in a Google Cloud Storage bucket.

    This tool leverages the native multimodal capabilities of Gemini to read
    a PDF directly from a GCS URI and extracts the product number.

    Args:
        gcs_uri: The GCS URI of the PDF file to summarize.
                 (e.g., "gs://your-bucket-name/your-file.name.pdf")

    Returns:
        A string only the product number.
    """
 
    model = GenerativeModel("gemini-2.5-flash")

    # Create a Part object directly from the GCS URI
    pdf_file = Part.from_uri(
        mime_type="application/pdf",
        uri=gcs_uri
    )

    # The prompt for the model
    prompt = "Extract the product number from the document. Retrun only the product number."

    # Construct the full request and generate the content
    response = model.generate_content([pdf_file, prompt])

    return  {"status": "success", "Product": response.text}


In [25]:
extract_product_no('gs://cob-agent-sandbox/schreiber_test/sample_coa.pdf')

/Users/carolinebonnes/github/adk/schreiber-adk/env/lib/python3.12/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


{'status': 'success', 'summary': 'ES788'}

In [2]:
def determine_hold(analysis):
    """
    Reviews the analysis of a COA document against it's spec sheet.

    This tool leverages Gemini to review an analysis and determine if a product should go on hold. 

    Args:
        analysis: The analysis to review.

    Returns:
        A string containing a `True` or `False ` response. 
    """

    model = GenerativeModel("gemini-2.5-flash")

    prompt = f"Review the provided analysis. If any metrics do not conform, or are out of range, return True. \
    If all metrics conform to specifications, and all required metrics are provided, return False.\
    Analysis: {analysis}"
    
    response = model.generate_content([prompt]).text.strip().lower()

    print(response.lower)
    if response=='true': 
        return  {"status": "success", "summary": 'The product does not conform and should be put on hold.'}
    else:
        return  {"status": "success", "summary": "The product conforms to all specs."}

In [6]:
analysis = """The provided Certificate of Analysis (COA) for Product No.: ES788 aligns perfectly with the requirements outlined in the Spec Sheet for the same product. Here's a breakdown of the comparison: **1. Specification Requirements vs. COA Requirements:** * **Appearance:** * **Spec Sheet Requirement:** "Clear, colorless, no observable particulate" * **COA Lot #2234008 Requirement:** "Clear, colorless, no observable particulate" * **COA Lot #2313621 Requirement:** "Clear, colorless, no observable particulate" * **Match:** Perfect match. * **Color (APHA):** * **Spec Sheet Requirement:** "10 Max" * **COA Lot #2234008 Requirement:** "10 Max" * **COA Lot #2313621 Requirement:** "10 Max" * **Match:** Perfect match. * **Specific Gravity @20°C:** * **Spec Sheet Requirement:** "0.9114-0.9246 @20°C" * **COA Lot #2234008 Requirement:** "0.9114-0.9246 @20°C" * **COA Lot #2313621 Requirement:** "0.9114-0.9246 @20°C" * **Match:** Perfect match. **2. COA Results vs. Requirements:** * **Lot #2234008:** * **Appearance:** "Conforms" (meets "Clear, colorless, no observable particulate") * **Color (APHA):** "0" (meets "10 Max") * **Specific Gravity @20°C:** "0.9200" (falls within "0.9114-0.9246") * **Conclusion for Lot #2234008:** All results meet the specified requirements. * **Lot #2313621:** * **Appearance:** "Colorless Liquid" (meets "Clear, colorless, no observable particulate") * **Color (APHA):** "0" (meets "10 Max") * **Specific Gravity @20°C:** "0.920" (falls within "0.9114-0.9246") * **Conclusion for Lot #2313621:** All results meet the specified requirements. **Summary:** Both lots (2234008 and 2313621) detailed in the Certificate of Analysis (COA) demonstrate full compliance with the specification requirements outlined in the Spec Sheet for Product No.: ES788. The "Requirements" listed in the COA are identical to those in the Spec Sheet, and the "Results" for both lots successfully meet these established criteria."""
determine_hold(analysis)

/Users/carolinebonnes/github/adk/schreiber-adk/env/lib/python3.12/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


<built-in method lower of str object at 0x156c0bbd0>


{'status': 'success', 'summary': 'The product conforms to all specs.'}

In [14]:
def update_database():

    storage_client = storage.Client()
    bucket = storage_client.bucket('cob-agent-sandbox')
    blob = bucket.blob('schreiber_test/product_holds.txt')

    product = ''

    new_line = f'Product #{product} status: Hold'

    existing_content = blob.download_as_text()

    new_content = existing_content + "\n" + new_line

    blob.upload_from_string(new_content, content_type='text/plain')
